In [40]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from proj1_helpers import *
from imple import *
from plots import *

## Load the training data into feature matrix, class labels, and event ids:

In [5]:
import zipfile 

In [6]:
zf_train = zipfile.ZipFile('data/train.csv.zip') 
zf_train.extractall()
zf_test = zipfile.ZipFile('data/test.csv.zip')
zf_test.extractall()

In [7]:
from proj1_helpers import *
#PATH1='..\'
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [29]:
def select_good_features(x_, ids):
    """ return data that is contained in the ids list. """
    return x_[:, ids]

def remove_missing_features(x_, threshold=0.25):
    """Returns a list of booleans (one bool per column) indicating whether the data is "good" (has less NaNs than a threshold) """
    nans_percentage = np.mean(x_ == -999, axis=0)
    return (nans_percentage < threshold)

#select_good_features(tX, ids)
missing_values= remove_missing_features(tX)
x_train = select_good_features(tX, missing_values)
#x_test = select_good_features(x_test, missing_ids)

In [30]:
x_train.shape
#x_train= np.delete(x_train,missing_values, axis=1)
#x_test= np.delete(x_test,cat_feat_id,axis=1)

(250000, 20)

In [38]:
y_train=y

In [ ]:
#Cross validation on ridge regression
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

def cross_validation_ridge(y, x, k_indices, k,degree,lambda_):
    """return loss CV on a degree of a polynome."""

    te_index = k_indices[k]
    tr_index = k_indices[~(np.arange(k_indices.shape[0])==k)]
    tr_index = tr_index.reshape(-1)

    x_te = x[te_index]
    y_te = y[te_index]
    x_tr = x[tr_index]
    y_tr = y[tr_index]
    # ***************************************************
    #build polynome with corresponding degree to the corresponding feature identified by id_
    #least squares on tx_tr
    
    x_train_pol = build_poly(x_tr[:,:19],degree)
    tx_tr = np.concatenate((x_train_pol,x_tr[:,19:]),axis = 1)
    x_test_pol = build_poly(x_te[:,:19],degree)
    tx_te = np.concatenate((x_test_pol,x_te[:,19:]),axis =1)
    
    w, loss_tr = ridge_regression(y_tr,tx_tr,lambda_)

    loss_tr = np.sqrt(2 * loss_tr)
    loss_te = np.sqrt(2 * compute_loss(y_te,tx_te,w))
    return w,loss_tr, loss_te


In [46]:
k_fold = 3
seed = 4
degree = 8

lambdas_ = np.linspace(2,4,40) #Lamdba for best submission : linspace(2,4,40)

losses_tr_ridge = []
losses_te_ridge = []
# split data in k fold
k_indices = build_k_indices(y_train, k_fold, seed)

for lambda_ in lambdas_:
    loss_tr_k = []
    loss_te_k = []
    for k in range(0,k_fold):
        w,loss_tr,loss_te = cross_validation_ridge(y_train, x_train, k_indices, k,degree, lambda_)
        loss_tr_k.append(loss_tr)
        loss_te_k.append(loss_te)
    losses_tr_ridge.append(np.mean(loss_tr_k))
    losses_te_ridge.append(np.mean(loss_te_k))

In [62]:
min(losses_te_ridge)

2997958.2898170487

In [47]:
best_lambda = lambdas_[losses_te_ridge.index(min(losses_te_ridge))]
print(best_lambda)

4.0


In [51]:
initial_w=np.zeros((x_train.shape[1], 1))
max_iters=50
gamma=0.5

reg_logistic_regression(y_train, x_train, best_lambda , initial_w, max_iters, gamma)

MemoryError: Unable to allocate 466. GiB for an array with shape (250000, 250000) and data type float64

In [103]:


def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

def cross_validation(y, x, k_indices, k, lambda_, initial_w, max_iters, gamma):
    """return the loss of ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # get k'th subgroup in test, others in train: TODO
    index_te = k_indices[k]
    index_tr = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    index_tr = index_tr.reshape(-1)
    x_te=x[index_te]
    y_te=y[index_te]
    x_tr=x[index_tr]
    y_tr=y[index_tr]
    losses_tr, w=logistic_regression(y_tr, x_tr, initial_w, max_iters, gamma)
    print(w.shape)
    losses_te, w_te=logistic_regression(y_te, x_te, initial_w, max_iters, gamma)
    
    return losses_tr, losses_te


In [ ]:
degree= 1
lambda_=0.01

for i in range (1, 20):
    k_indices= build_k_indices(y, i, seed=2)
    loss_tr, loss_te =cross_validation(y, tX, k_indices, i, lambda_, degree)
    print('training loss', loss_tr, 'test loss', loss_te)


In [113]:
seed = 1
degree = 30
k_fold = 4
lambdas = np.logspace(-4, 0, 30)

initial_w = np.array([100, 10])
max_iters = 50
gamma = 0.1

# split data in k fold
k_indices = build_k_indices(y_data, k_fold, seed)
# define lists to store the loss of training data and test data
rmse_tr = []
rmse_te = []
# cross validation: TODO
for lambda_ in lambdas:
    rmse_tr_tmp = []
    rmse_te_tmp = []
    for k in range(k_fold):
        loss_tr, loss_te = cross_validation(y_data[:100], x_data[:100], k_indices, k, lambda_, initial_w, max_iters, gamma)
        rmse_tr_tmp.append(loss_tr)
        rmse_te_tmp.append(loss_te)
    rmse_tr.append(np.mean(rmse_tr_tmp))
    rmse_te.append(np.mean(rmse_te_tmp))   

ValueError: shapes (75,30) and (2,) not aligned: 30 (dim 1) != 2 (dim 0)

In [119]:
loss, w = least_squares(y[:30], tX[:30])

(30, 1)
(30, 1)
(30, 30)


In [115]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [120]:
OUTPUT_PATH = 'sample_submission.csv' # TODO: fill in desired name of output file for submission
weights=w

y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)